### 🔧 Alustukset ja asetukset

In [ ]:
from ultralytics import YOLO
import torch
import cv2
import matplotlib.pyplot as plt
import os
import glob
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from torch import nn
from torch.optim import Adam
from sklearn.metrics import classification_report, confusion_matrix
from PIL import Image
import numpy as np
%matplotlib inline
import seaborn as sns
import numpy as np
from ultralytics.utils import LOGGER
import logging



### 🧠 YOLO-mallin koulutus

In [ ]:


# GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
LOGGER.setLevel(logging.ERROR)

# 📦 Lataa malli
model = YOLO('yolo11x.pt')
model.to(device)

# 🧪 Kouluta malli suoraan parametreilla (ei cfg)
model.train(
    data='config/materials.yaml',
    epochs=30,
    imgsz=640,
    batch=8,
    patience=10,
    cache=False, 
    project='runs/train',
    name='materials_test_direct',
    exist_ok=True,
    lr0=0.005,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    box=0.03,
    cls=0.4,
    dfl=1.5,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,
    translate=0.1,
    scale=0.5,
    shear=1-2,
    perspective=0.002,
    flipud=0.0,
    fliplr=0.5,
    mosaic=0.1,
    mixup=0.1,
    copy_paste=0.0
)

# 🧹 Tyhjennä mahdollinen ylimääräinen muisti
torch.cuda.empty_cache()

# 🔍 Validoi malli
model = YOLO('runs/train/materials_test_direct/weights/best.pt')
model.to(device)
results = model.val(augment=True) #Test Time Augmentation

# 🧹 Tyhjennä mahdollinen ylimääräinen muisti
torch.cuda.empty_cache()

# 🧾 Kokonaistulokset
print("\n✅ YOLO-validointi:")
print(f"mAP@0.5:      {results.box.map50:.4f}")
print(f"mAP@0.5:0.95: {results.box.map:.4f}\n")

# 🔍 Luokkakohtaiset mittarit
ap_list = results.box.ap
precisions = results.box.p
recalls = results.box.r
class_names = list(results.names.values())

print("📊 Luokkakohtaiset tulokset:\n")
for i, class_id in enumerate(list(results.names.keys())[:len(ap_list)]):
    name = results.names[class_id]
    print(f"Class '{name:20s}': Precision = {precisions[i]:.4f}, Recall = {recalls[i]:.4f}, AP = {ap_list[i]:.4f}")



In [ ]:
# 📊 Confusion matrix 
if hasattr(results, "confusion_matrix"):
    cm = results.confusion_matrix.matrix
    class_names = list(results.names.values())

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt=".0f", cmap="Blues",
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("YOLO Confusion Matrix")
    plt.tight_layout()
    plt.show()
else:
    print("\n⚠️ Confusion matrix ei ole saatavilla.")

### ✅ Tarkista validointikuvien labelit

In [ ]:
image_dir = 'datasets/materials/images/validation'
label_dir = 'datasets/materials/labels/validation'

image_files = sorted(glob.glob(f'{image_dir}/*.jpg')) + sorted(glob.glob(f'{image_dir}/*.png'))
missing_labels = [os.path.basename(f) for f in image_files
                  if not os.path.exists(os.path.join(label_dir, os.path.splitext(os.path.basename(f))[0] + '.txt'))]

if missing_labels:
    print("⚠️ Näiltä kuvilta puuttuu label:")
    for f in missing_labels:
        print("-", f)
else:
    print("✅ Kaikille kuville löytyy labelit.")


### 🔍 Ennusteet validointikuville + visualisointi

In [ ]:
trained_model = YOLO('runs/train/materials_test_direct/weights/best.pt')
results = trained_model(image_files[:6])

fig, axes = plt.subplots(2, 3, figsize=(18, 10))  # 2 riviä, 3 saraketta
axes = axes.flatten()  # helpottaa indeksin käyttöä

for i, r in enumerate(results):
    img_rgb = cv2.cvtColor(r.plot(), cv2.COLOR_BGR2RGB)
    axes[i].imshow(img_rgb)
    axes[i].set_title(os.path.basename(image_files[i]))
    axes[i].axis('off')

# Jos vähemmän kuvia kuin paikkoja (varmuuden vuoksi)
for j in range(len(results), len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()


### ✂️ Croppaa kuvat YOLO-labelien mukaan

In [ ]:
def generate_cropped_images(image_root, label_root, output_root):
    os.makedirs(output_root, exist_ok=True)
    image_paths = sorted(glob.glob(os.path.join(image_root, '*.jpg'))) + \
                  sorted(glob.glob(os.path.join(image_root, '*.png')))
    for img_path in image_paths:
        name = os.path.splitext(os.path.basename(img_path))[0]
        label_path = os.path.join(label_root, f"{name}.txt")
        if not os.path.exists(label_path): continue
        img = cv2.imread(img_path)
        if img is None: continue
        h, w = img.shape[:2]
        with open(label_path, 'r') as f:
            for idx, line in enumerate(f):
                class_id, xc, yc, bw, bh = map(float, line.strip().split())
                x1 = max(0, int((xc - bw / 2) * w))
                y1 = max(0, int((yc - bh / 2) * h))
                x2 = min(w, int((xc + bw / 2) * w))
                y2 = min(h, int((yc + bh / 2) * h))
                cropped = img[y1:y2, x1:x2]
                if cropped.size == 0: continue
                out_path = os.path.join(output_root, str(int(class_id)), f"{name}_{idx}.jpg")
                os.makedirs(os.path.dirname(out_path), exist_ok=True)
                cv2.imwrite(out_path, cropped)

for split in ['train', 'validation']:
    generate_cropped_images(f'datasets/materials/images/{split}', f'datasets/materials/labels/{split}', f'datasets/materials/cropped/{split}')


### 🏋️ DenseNet-koulutus

In [ ]:

# 🔄 Transformit
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 📂 Datasetit ja DataLoaderit
train_data = datasets.ImageFolder('datasets/materials/cropped/train', transform=transform)
val_data = datasets.ImageFolder('datasets/materials/cropped/validation', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

# 🧠 Malli: DenseNet121 esikoulutettuna
model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
model.classifier = nn.Linear(model.classifier.in_features, len(train_data.classes))
model = model.to(device)

# ⚙️ Koulutuksen asetukset
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-4)

# 📊 Koulutushistorian seurantamuuttujat
train_losses, val_accuracies = [], []
best_val_acc = 0.0

# 🏋️‍♂️ Koulutussilmukka
for epoch in range(25):
    model.train()
    train_correct = 0
    train_total = 0
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_acc = 100 * train_correct / train_total
    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)

    # 🔍 Validointi
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = 100 * val_correct / val_total
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1}: Train Acc = {train_acc:.2f}%, Val Acc = {val_acc:.2f}%")

    # 💾 Tallenna paras malli
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'densenet_materials_best.pth')

# 📈 Visualisointi: Loss ja Accuracy
epochs = range(1, len(train_losses) + 1)
plt.figure(figsize=(12, 5))

# 🎯 Validation Accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs, val_accuracies, marker='o', label='Val Accuracy', color='green')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Validation Accuracy over Epochs')
plt.grid(True)
plt.legend()

# 📉 Training Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, train_losses, marker='o', label='Train Loss', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()


### 🤖 Hybridiluokitus validointikuville

In [ ]:
from collections import Counter

class_names = train_data.classes
yolo_model = YOLO('runs/train/materials_test_direct/weights/best.pt')
model.eval()

preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

validation_images = glob.glob('datasets/materials/images/validation/*.jpg') + \
                    glob.glob('datasets/materials/images/validation/*.png')

results_counter = Counter()

for img_path in validation_images:
    img = cv2.imread(img_path)
    if img is None:
        continue

    results = yolo_model(img_path)[0]

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        yolo_cls = int(box.cls[0])
        yolo_conf = float(box.conf[0])

        crop = img[y1:y2, x1:x2]
        if crop.shape[0] == 0 or crop.shape[1] == 0: # korvattu crop.size == 0:
            continue

        pil_img = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
        input_tensor = preprocess(pil_img).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            densenet_cls = output.argmax(dim=1).item()
            densenet_conf = torch.softmax(output, dim=1)[0][densenet_cls].item()

        # Valitaan YOLO tai DenseNet
        if yolo_conf >= 0.75:
            final_cls = yolo_cls
            source = "YOLO"
        else:
            final_cls = densenet_cls
            source = "DenseNet"

        results_counter[(class_names[final_cls], source)] += 1

# 🧾 Tulosta yhteenveto
print("\n🔍 Luokitusyhteenveto (hybridimalli, valinta YOLO vs DenseNet):\n")
for (cls_name, source), count in sorted(results_counter.items()):
    print(f"{cls_name:25s} [{source}]: {count} kpl")


### 📊 DenseNetin suorituskykymittarit

In [ ]:
val_data = datasets.ImageFolder('datasets/materials/cropped/validation', transform=transform)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

y_true, y_pred = [], []
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("\n✅ DenseNet tarkkuusvalidaatio:")
print(classification_report(y_true, y_pred, target_names=val_data.classes, zero_division=0))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))


## 🧠 **Vertailufunktio**

- Lataa yhden kuvan polusta image_path
- Käyttää YOLO-mallia havaitakseen kohteet ja piirtää ne kuvaan

Jokaisen havaintoboksin kohdalla:
- YOLO antaa luokan ja todennäköisyyden → piirretään keltaisella
- DenseNet luokittelee saman alueen → piirretään vihreällä
- Hybridisääntö: jos YOLO:n varmuus ≥ 75 %, käytetään YOLOa, muuten DenseNetiä → piirretään laatikko ja label, jossa näkyy kumpaa mallia käytettiin

Tarkkuudet (%) näytetään laatikon yläpuolella

Näyttää kolme kuvaa rinnakkain:
- YOLO-tulokset
- DenseNet-tulokset
- Hybridimallin tulokset

In [ ]:

def show_yolo_vs_densenet(image_path):

    
    # Asetukset
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

    # Luokat
    sample_dataset = datasets.ImageFolder("datasets/materials/cropped/train", transform=transform)
    class_names = sample_dataset.classes

    # Mallit
    yolo_model = YOLO("runs/train/materials_test_direct/weights/best.pt")
    densenet_model = models.densenet121(weights=None)
    densenet_model.classifier = torch.nn.Linear(densenet_model.classifier.in_features, len(class_names))
    densenet_model.load_state_dict(torch.load("densenet_materials_5class.pth", map_location=device))
    densenet_model.to(device)
    densenet_model.eval()

    # Kuva
    img = cv2.imread(image_path)
    h, w = img.shape[:2]

    # Valmistele kopiot
    yolo_img = img.copy()
    densenet_img = img.copy()
    hybrid_img = img.copy()

    # YOLO ennusteet
    #results = yolo_model(image_path)[0] piirtää kaikki ennustetut ja tallennetut bounding boxit
    results = yolo_model.predict(image_path, verbose=False)[0] # käynnistää uuden puhtaan ennusteen ja verbose=False estää turhan tulostamisen.


    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        # YOLO
        yolo_cls = int(box.cls[0])
        yolo_conf = float(box.conf[0])
        yolo_label = f"{class_names[yolo_cls]} ({yolo_conf*100:.1f}%)"
        cv2.rectangle(yolo_img, (x1, y1), (x2, y2), (0, 255, 255), 2)
        # Labelin määrittely
        label = f"{class_names[yolo_cls]} ({yolo_conf*100:.1f}%)"
        
        # Tekstin koko ja sijainti
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1.5
        thickness = 4
        (text_w, text_h), _ = cv2.getTextSize(label, font, font_scale, thickness)
        text_x = x1 + (x2 - x1 - text_w) // 2
        text_y = max(y1 - 10, text_h + 10)  # ettei mene kuvan ulkopuolelle
        
        # Piirrä laatikko + teksti
        cv2.rectangle(yolo_img, (x1, y1), (x2, y2), (0, 255, 255), 2)
        cv2.putText(yolo_img, label, (text_x, text_y), font, font_scale, (0, 255, 255), thickness)


        # DenseNet
        crop = densenet_img[y1:y2, x1:x2]
        if crop.size == 0:
            continue
        pil_img = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
        input_tensor = transform(pil_img).unsqueeze(0).to(device)

        with torch.no_grad():
            output = densenet_model(input_tensor)
            pred_cls = output.argmax(dim=1).item()
            dn_conf = torch.softmax(output, dim=1)[0][pred_cls].item()

        dn_label = f"{class_names[pred_cls]} ({dn_conf*100:.1f}%)"
        cv2.rectangle(densenet_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(densenet_img, dn_label, (text_x, text_y), font, font_scale, (0, 255, 0), thickness)


        # Hybrid: YOLO if conf >= 0.75 else DenseNet
        if yolo_conf >= 0.75:
            final_cls = yolo_cls
            final_conf = yolo_conf
            source = "YOLO"
            color = (0, 255, 255)
        else:
            final_cls = pred_cls
            final_conf = dn_conf
            source = "DenseNet"
            color = (0, 255, 0)

        label = f"{class_names[final_cls]} ({final_conf*100:.1f}%) [{source}]"
        cv2.rectangle(hybrid_img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(hybrid_img, label, (text_x, text_y), font, font_scale, color, thickness)

    # Näytetään kolme kuvaa rinnakkain
    plt.figure(figsize=(18, 6))
    plt.subplot(1, 3, 1)
    plt.imshow(cv2.cvtColor(yolo_img, cv2.COLOR_BGR2RGB))
    plt.title("YOLO-luokitus")
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.imshow(cv2.cvtColor(densenet_img, cv2.COLOR_BGR2RGB))
    plt.title("DenseNet-luokitus")
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(cv2.cvtColor(hybrid_img, cv2.COLOR_BGR2RGB))
    plt.title("Hybridiluokitus")
    plt.axis("off")

    plt.tight_layout()

    #class_names = ["Betoni", "Ei materiala", "Materiaali ei tiedossa", "Muovi", "Teräs"]

    # Tulosta luokkien selitykset (esim. 0 = Betoni jne)
    print("\n📘 Luokkakoodien selitykset:")
    for idx, name in enumerate(class_names):
        print(f"{idx} = {name}")

    plt.show()


### Haetaan yksi tietty kuva

In [ ]:
# kutsu
show_yolo_vs_densenet("datasets/materials/images/validation/176137_1.jpg")


### Kutsutaan random kolme kuvaa

In [ ]:
import glob
import random

# Kaikki validointikuvat
all_images = glob.glob("datasets/materials/images/validation/*.jpg") + \
             glob.glob("datasets/materials/images/validation/*.png")

# Poistetaan jo käytetyt kuvat
used_images = [
    "datasets/materials/images/validation/174114_2.jpg"
]
remaining_images = list(set(all_images) - set(used_images))

# Arvotaan 3 satunnaista kuvaa
random_images = random.sample(remaining_images, 3)

# Ajetaan funktio satunnaisille kuville
for path in random_images:
    show_yolo_vs_densenet(path)
